In [14]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from csv import writer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, Conv1D,MaxPooling1D,BatchNormalization
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib
from matplotlib import ticker as ticker
import matplotlib.pyplot as plt
import h5py
import os
from sklearn.metrics import f1_score
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
import json
from sklearn.model_selection import StratifiedShuffleSplit

In [15]:
data = json.load(open("flipStress_p12_t0.5s.json"))
trainX = np.array(data['X'],dtype=np.float32)
trainY= np.array(data['y'])

In [16]:
trainX = np.expand_dims(trainX,axis=2)

In [17]:
print(trainX.shape)

(200, 5000, 1)


In [18]:
input_data = trainX

minimum = np.amin(input_data)
maximum = np.amax(input_data)

input_data_norm = (input_data-minimum)/(maximum-minimum)

print(input_data_norm.shape)

input_data_norm = input_data_norm

(200, 5000, 1)


In [19]:
trainY= keras.utils.to_categorical(trainY)
print(trainY.shape)

(200, 2)


In [20]:
def model_create(x=None):
    numberOfWebsite=2

    input = keras.Input(shape = (5000,1))
    x = layers.LSTM(32,input_shape=(input.shape[1:]), return_sequences=True)(input)

    x = layers.Conv1D(256, 32, activation="relu", padding="same")(x)
    x = MaxPooling1D(pool_size=4)(x)


    x = layers.Conv1D(256, 32, activation="relu", padding="same")(x)
    x = MaxPooling1D(pool_size=4)(x)

    x = layers.Dropout(0.7)(x)

    x = layers.Flatten()(x)

    x = Dense(numberOfWebsite,activation='softmax')(x)

    model = keras.Model(inputs = input, outputs = x)

    return model

In [21]:
model = model_create()
model.summary()

Model: "functional_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_23 (InputLayer)          │ (None, 5000, 1)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_23 (LSTM)                       │ (None, 5000, 32)            │           4,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_46 (Conv1D)                   │ (None, 5000, 256)           │         262,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_46 (MaxPooling1D)      │ (None, 1250, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_47 (Conv1D)                   │ (None, 1250, 256)           │       2,097,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_47 (MaxPooling1D)      │ (None, 312, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_23 (Dropout)                 │ (None, 312, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_23 (Flatten)                 │ (None, 79872)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 2)                   │         159,746 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,523,906 (9.63 MB)

 Trainable params: 2,523,906 (9.63 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
n_splits = 10
sss = StratifiedShuffleSplit(n_splits=n_splits, random_state=42)
all_accuracies = []
split_num = 1

for train_index, val_index in sss.split(input_data_norm, trainY.argmax(axis=1)):

    X_train_fold, X_val_fold = input_data_norm[train_index], input_data_norm[val_index]
    y_train_fold, y_val_fold = trainY[train_index], trainY[val_index]

    # Print the shapes of the training and validation sets
    print(f"Split {split_num}:")
    print(f"Training set shape: {X_train_fold.shape}")
    print(f"Validation set shape: {X_val_fold.shape}")

    # Create a new model for each split
    model = model_create()

   #Compiling the network
    model.compile( loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.001, decay=1e-6), metrics=['accuracy'])

    # Train the model
    history = model.fit(
        X_train_fold, y_train_fold,
        epochs=40,
        validation_data=(X_val_fold, y_val_fold),
        verbose=0
    )

    # Evaluate the model on the validation data
    val_loss, val_accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)

    all_accuracies.append(val_accuracy)
    print(f"Split {split_num} accuracy: {val_accuracy:.4f}\n")


    split_num += 1

# Calculate the average accuracy and standard deviation over all splits
average_accuracy = np.mean(all_accuracies)
std_accuracy = np.std(all_accuracies)
print(f"Average accuracy over {n_splits} splits: {average_accuracy:.4f}")
print(f"Standard deviation over {n_splits} splits: {std_accuracy:.4f}")

Split 1:
Training set shape: (180, 5000, 1)
Validation set shape: (20, 5000, 1)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Split 1 accuracy: 0.7000

Split 2:
Training set shape: (180, 5000, 1)
Validation set shape: (20, 5000, 1)
Split 2 accuracy: 0.3500

Split 3:
Training set shape: (180, 5000, 1)
Validation set shape: (20, 5000, 1)
Split 3 accuracy: 0.6000

Split 4:
Training set shape: (180, 5000, 1)
Validation set shape: (20, 5000, 1)
Split 4 accuracy: 0.5000

Split 5:
Training set shape: (180, 5000, 1)
Validation set shape: (20, 5000, 1)
Split 5 accuracy: 0.7000

Split 6:
Training set shape: (180, 5000, 1)
Validation set shape: (20, 5000, 1)
Split 6 accuracy: 0.5000

Split 7:
Training set shape: (180, 5000, 1)
Validation set shape: (20, 5000, 1)
Split 7 accuracy: 0.6500

Split 8:
Training set shape: (180, 5000, 1)
Validation set shape: (20, 5000, 1)
Split 8 accuracy: 0.7000

Split 9:
Training set shape: (180, 5000, 1)
Validation set shape: (20, 5000, 1)
Split 9 accuracy: 0.5500

Split 10:
Training set shape: (180, 5000, 1)
Validation set shape: (20, 5000, 1)
Split 10 accuracy: 0.5000

Average accuracy o